# Setup for the sample
**Pre-requisites**:
- an active MLflow tracking server managed by Amazon SageMaker
- the IAM role associated with the notebook must have sufficient permissions to write to the MLflow tracking server
- Access via Amazon Bedrock to Anthropic Clause v3 Haiku and Sonnet

In [2]:
# !pip install --upgrade sagemaker --quiet

In [1]:
import json

import sagemaker
from ipywidgets import interactive

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()
sagemaker_client = sagemaker_session.sagemaker_client

In [3]:
def get_studio_username() -> str | None:
    """Attempt tog get user name from Studio environment"""
    try:
        from sagemaker.session import NOTEBOOK_METADATA_FILE

        with open(NOTEBOOK_METADATA_FILE, "rb") as f:
            nb_metadata = json.load(f)
        if user_name := nb_metadata.get("UserProfileName"):
            return user_name
        return sagemaker_client.describe_space(
            DomainId=nb_metadata.get("DomainId"),
            SpaceName=nb_metadata.get("SpaceName"),
        )["OwnershipSettings"]["OwnerUserProfileName"]
    except Exception as e:
        print(e)
        return

In [4]:
tracking_servers_list = {
    k.get("TrackingServerName"): k.get("TrackingServerArn")
    for k in sagemaker_client.list_mlflow_tracking_servers(
        # TrackingServerStatus='Started'
    )["TrackingServerSummaries"]
}

select_tracking_server = interactive(lambda x: x, x=tracking_servers_list)
select_tracking_server

interactive(children=(Dropdown(description='x', options={'sagemaker-fmeval-workshop': 'arn:aws:sagemaker:us-we…

## Write .env file

In [5]:
with open(".env", "w") as f:
    f.write(f"MLFLOW_TRACKING_USERNAME={get_studio_username()}\n")
    f.write(f"MLFLOW_TRACKING_URI={select_tracking_server.result}\n")

In [6]:
!cat .env

MLFLOW_TRACKING_USERNAME=amazonq-br-demo
MLFLOW_TRACKING_URI=arn:aws:sagemaker:us-west-2:461312420708:mlflow-tracking-server/sagemaker-fmeval-workshop


## Install dependencies

We install `fmeval-mlflow` directly from the repository.

In [7]:
%pip install ..

Processing /home/sagemaker-user/fmeval-evaluation-using-mlflow
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fmeval-1.2.2-py3-none-any.whl.metadata (14 kB)
  Using cached aiohttp-3.11.14-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached grpcio-1.71.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached jiwer-3.1.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached mypy_boto3_bedrock-1.37.18-py3-none-any.whl.metadata (16 kB)
  Using cached pyfunctional-1.5.0-py3-none-any.whl.metadata (40 kB)
  Using cached ray-2.40.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached

And then install additional dependencies necessary for running the examples notebooks.

In [8]:
%pip install -r requirements.txt -U

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached mlflow-2.16.2-py3-none-any.whl.metadata (29 kB)
  Using cached ray-2.44.1-cp311-cp311-manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached mlflow_skinny-2.16.2-py3-none-any.whl.metadata (30 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
Using cached mlflow-2.16.2-py3-none-any.whl (26.7 MB)
Using cached mlflow_skinny-2.16.2-py3-none-any.whl (5.6 MB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Using cached ray-2.44.1-cp311-cp311-manylinux2014_x86_64.whl (68.1 MB)
Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (14